In [6]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import save_model, load_model
import numpy as np
import os

# Load mô hình VGG16 (không bao gồm lớp fully connected layer ở cuối)
base_model = VGG16(weights='imagenet', include_top=False)

# Xây dựng mô hình mới chỉ với các lớp đầu tiên của VGG16 để trích xuất đặc trưng
model = Model(inputs=base_model.input, outputs=base_model.layers[5].output)  # Đây là lớp Conv2D thứ 4 trong mô hình VGG16

# Đường dẫn tới thư mục chứa ảnh huấn luyện
train_dir = 'Black2/train'

# Tạo một list để lưu trữ các đặc trưng và nhãn tương ứng
features = []
labels = []

# Lặp qua từng lớp trong thư mục huấn luyện và trích xuất đặc trưng từ hình ảnh
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        img = image.load_img(img_path, target_size=(100, 100))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        features.append(feature.flatten())
        labels.append(class_name)

# Chuyển features và labels thành numpy arrays
features = np.array(features)
labels = np.array(labels)

# Lưu các đặc trưng và nhãn
np.save('extracted_features.npy', features)
np.save('labels.npy', labels)

# Lưu mô hình đã chỉnh sửa thành một tệp .h5
save_model(model, 'feature_extractor_model.h5')

# Tải lại mô hình từ tệp .h5
feature_extractor = load_model('feature_extractor_model.h5')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━

OSError: [Errno 22] Invalid argument: 'extracted_features.npy'

In [4]:
import os
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load các đặc trưng đã trích xuất từ mô hình VGG16
train_features = np.load('extracted_features.npy', mmap_mode='r')
labels = np.load('labels.npy', mmap_mode='r')

# Load mô hình feature_extractor
feature_extractor = load_model('feature_extractor_model.h5')

def find_similar_images():
    # Load đặc trưng của ảnh đầu vào
    input_image_path = selected_image_path.get()
    input_image = image.load_img(input_image_path, target_size=(200, 200))
    input_image = image.img_to_array(input_image)
    input_image = np.expand_dims(input_image, axis=0)
    input_feature = feature_extractor.predict(input_image)

    # Tính toán độ tương đồng giữa ảnh đầu vào và tất cả các ảnh trong tập huấn luyện
    similarities = []
    for train_feature in train_features:
        similarity = np.dot(input_feature.flatten(), train_feature.flatten())
        similarities.append(similarity)

    # Lựa chọn ra 3 ảnh có độ tương đồng cao nhất
    top_indices = np.argsort(similarities)[-3:][::-1]

    # Hiển thị các ảnh tương đồng trong giao diện
    similar_images = []
    for index in top_indices:
        similar_image_path = os.path.join(train_dir, labels[index])  # Đường dẫn tới ảnh tương đồng
        similar_image = Image.open(similar_image_path)
        similar_image = similar_image.resize((200, 200))
        similar_images.append(ImageTk.PhotoImage(similar_image))

    # Hiển thị các ảnh tương đồng trong giao diện
    for i, similar_img in enumerate(similar_images):
        similar_image_label = tk.Label(app, image=similar_img)
        similar_image_label.image = similar_img
        similar_image_label.pack(side=tk.LEFT, padx=5)

app = tk.Tk()
app.title("Fruit Classifier")

# Tổ chức giao diện và các chức năng khác giữ nguyên

app.mainloop()


ValueError: Layer node index out of bounds.
inbound_layer = <Conv2D name=conv2d_3, built=True>
inbound_layer._inbound_nodes = [<Node operation=<Conv2D name=conv2d_3, built=True>, id=2344275122064>]
inbound_node_index = 7